In [1]:
from desdeo_emo.EAs.RVEA import RVEA
from desdeo_problem.problem.Variable import variable_builder
from desdeo_problem.testproblems.TestProblems import test_problem_builder
from desdeo_problem.surrogatemodels.SurrogateModels import GaussianProcessRegressor
from desdeo_problem import ExperimentalProblem
from desdeo_tools.utilities.model_management import ikrvea_mm
import sys 
sys.path.append("NumPy_path") 
import numpy as np
import pandas as pd

from sklearn.gaussian_process.kernels import Matern
from pymoo.factory import get_problem, get_reference_directions
import copy

# we define the problem to be solved

In this example we have used the pymoo package to define dtlz2 problem with 3 objectives. Please not for now we cannot use the test_problem_maker class from DESDEO to use dtlz problems for online surrogate assisted methods and we need to define them manually.

In [2]:
def obj_function1(x):

    out = {
    "F": "",
    "G": "",
    }
    problem = get_problem("dtlz2", 12)
    problem._evaluate(x, out)
    return out['F'][:,0]
def obj_function2(x):

    out = {
    "F": "",
    "G": "",
    }
    problem = get_problem("dtlz2", 12)
    problem._evaluate(x, out)
    return out['F'][:,1]
def obj_function3(x):

    out = {
    "F": "",
    "G": "",
    }
    problem = get_problem("dtlz2", 12)
    problem._evaluate(x, out)
    return out['F'][:,2]

# Setting the parameters

In [3]:
number_of_obj = 3
number_of_var = number_of_obj +4

n_show = 5
archive = None
m_archive = None
refpoint = np.asarray([0.2,0.5,0.9])
n_obj = number_of_obj # number of objectives
n_var = number_of_var # number of variables
var_names = ["x" + str(i + 1) for i in range(n_var)]
obj_names = ["f" + str(i + 1) for i in range(n_obj)]
problem = test_problem_builder(
                name='DTLZ2', n_of_objectives=n_obj, n_of_variables=n_var
            )

Here we define the evolver and create the archive in pandas data frame type (datapd). Then we define the problem object.

## note:
currently we are using the ExperimentalProblem class. In the future updates of DESDEO this will change.

In [4]:
evolver = RVEA(
            problem, interact=True, n_iterations=1, n_gen_per_iter = 100)
    
    
initial_pop = copy.deepcopy(evolver.population.individuals)
initial_fitness = copy.deepcopy(evolver.population.fitness)
data = np.hstack((initial_pop, initial_fitness))
datapd = pd.DataFrame(data=data, columns=var_names+obj_names)
varsl = variable_builder(var_names,
        initial_values=[0] * n_var ,
        lower_bounds=[0.0] * n_var,
        upper_bounds=[1.0] * n_var
        )
problem = ExperimentalProblem(data = datapd, objective_names=obj_names, variable_names=var_names,\
     evaluators = [obj_function1, obj_function2, obj_function3])


## Training the surrogate models

In [5]:
problem.train(models=GaussianProcessRegressor, model_parameters={'kernel': Matern(nu=1.5)})


Defining the evolver with the surrogate models and some adjustments for interactive KRVEA.

The parameter u defines the number of solutions that need to be selected by the model management to evluate the original functions with them. 

In [6]:
evolver = RVEA(
            problem, interact=True, n_iterations=1, n_gen_per_iter = 100, use_surrogates= True)

pref, plot2 = copy.deepcopy(evolver.requests())
pref[2].response = copy.deepcopy(pd.DataFrame([refpoint],\
     columns=pref[2].content['dimensions_data'].columns))

u=10 #number of solutions per updates
i = 0

In [7]:
pref, plot2 = copy.deepcopy(evolver.requests())
pref[2].response = copy.deepcopy(pd.DataFrame([refpoint],\
    columns=pref[2].content['dimensions_data'].columns))
plot= evolver.iterate(pref[2])   
print(problem.evaluate(np.asarray([0.5,0.5,0.5,0.5,0.5,0.5,0.5]), use_surrogate= True))
problem = ikrvea_mm(refpoint, evolver, problem, u )
problem.train(models=GaussianProcessRegressor, model_parameters={'kernel': Matern(nu=1.5)})
print(problem.evaluate(np.asarray([0.5,0.5,0.5,0.5,0.5,0.5,0.5]), use_surrogate= True))


Evaluation Results Object 
Objective values are: 
[[0.51018956 0.49775215 0.6915514 ]]
Constraint violation values are: 
None
Fitness values are: 
[[0.51018956 0.49775215 0.6915514 ]]
Uncertainity values are: 
[[0.04847805 0.0537141  0.04608517]]

Evaluation Results Object 
Objective values are: 
[[0.50392414 0.49098729 0.7074659 ]]
Constraint violation values are: 
None
Fitness values are: 
[[0.50392414 0.49098729 0.7074659 ]]
Uncertainity values are: 
[[0.02331915 0.02579351 0.02206174]]



In [8]:
problem.archive[-u:]

,x1,x2,x3,x4,x5,x6,x7,f1,f2,f3
105,0.593056,0.570802,0.331713,0.531954,0.543793,0.550162,0.502528,0.384999,0.481799,0.829669
106,0.660095,0.580258,0.489166,0.546064,0.531300,0.455505,0.573335,0.315051,0.406501,0.869922
107,0.547868,0.633743,0.456410,0.532923,0.494344,0.583192,0.317884,0.370039,0.570613,0.790909
108,0.576021,0.663628,0.525693,0.442433,0.520535,0.500153,0.477523,0.312994,0.536194,0.790162
109,0.636347,0.701613,0.507931,0.456502,0.516869,0.445549,0.571506,0.246754,0.487331,0.849919
110,0.605544,0.815051,0.443791,0.470313,0.495888,0.630895,0.553163,0.170340,0.569746,0.833658
111,0.570202,0.617075,0.548282,0.565190,0.496961,0.322053,0.658755,0.376114,0.548003,0.830166
112,0.543471,0.632756,0.457067,0.533225,0.605036,0.435527,0.317804,0.376843,0.579137,0.792394
113,0.513355,0.683569,0.439726,0.565264,0.507352,0.678969,0.523172,0.343395,0.633015,0.751026
114,0.693316,0.748568,0.471672,0.520916,0.602744,0.551454,0.560437,0.181493,0.435391,0.902228


In [9]:
refpoint

array([0.2, 0.5, 0.9])

# Visualization
Here we visualize the last u solutions that interactive K-RVEA finds after each iteration

In [10]:
import plotly.graph_objs as go
from pymoo.factory import get_problem, get_reference_directions, get_visualization
#the next three lines are to get the true pareto front
p = get_problem("dtlz2", 12)
ref_dirs = get_reference_directions("das-dennis", 3, n_partitions=12)
pf = p.pareto_front(ref_dirs)

df = problem.archive[-u:]
x = df["f1"].to_numpy()
y = df["f2"].to_numpy()
z = df["f3"].to_numpy()

trace1 = go.Scatter3d(x=x, y=y, z=z, mode="markers",)
trace2 = go.Scatter3d(x=[refpoint[0]], y=[refpoint[1]], z=[refpoint[2]], mode="markers")
trace3 = go.Mesh3d(x=pf[:,0], y=pf[:,1], z=pf[:,2])
fig = go.Figure(data = [trace1, trace2, trace3])
fig.show()

# Results
The green mesh is the true Pareto front, the blue dots are the solutions that Interactive K-RVEA has found, and the red dot is the reference point.